In [1]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import numpy
import tflearn
import tensorflow as tf
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [5]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [6]:
with open('/content/gdrive/MyDrive/int.json') as file:
    data = json.load(file)

In [7]:
data

{'intents': [{'intent': 'feeling',
   'patterns': ['I am not feeling well',
    'I feel sick',
    'I am sick',
    'I have a headache',
    "I'm experiencing nausea",
    'I have a fever',
    'I am having fever',
    'My throat hurts',
    "I'm feeling tired",
    'I have a cough',
    'I have a stomach ache',
    'My back hurts',
    'I am experiencing cough',
    'I am not feeling good',
    'I am ill',
    'I feel ill',
    'My health is bad',
    'I am not well'],
   'responses': ["I'm sorry to hear that. Can you tell me more about your symptoms?",
    "I'm sorry you're not feeling well. Can you describe your symptoms for me?",
    'Sorry to hear that. Can you please provide me with more information about your symptoms?',
    "I'm sorry to hear that. Can you tell me where it hurts and what other symptoms you're experiencing?",
    "I'm here to help. Can you tell me more about your symptoms?"]},
  {'intent': 'back_pain',
   'patterns': ["My back has been hurting for a few days now

In [8]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()
# Each list to create
words = []
classes = []
doc_X = []
doc_y = []
# Loop through all the intents
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["intent"])
    
    # add the tag to the classes if it's not there already 
    if intent["intent"] not in classes:
        classes.append(intent["intent"])
# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [9]:
words

["'m",
 "'s",
 "'ve",
 'a',
 'ache',
 'achy',
 'activity',
 'after',
 'all',
 'alright',
 'also',
 'am',
 'an',
 'and',
 'any',
 'anything',
 'appointment',
 'appoitment',
 'are',
 'at',
 'ate',
 'available',
 'away',
 'back',
 'bad',
 'barely',
 'be',
 'because',
 'been',
 'body',
 'book',
 'bothering',
 'breath',
 'breathe',
 'but',
 'bye',
 'ca',
 'can',
 'causing',
 'chest',
 'chill',
 'concentrate',
 'congested',
 'congestion',
 'constant',
 'cough',
 'coughing',
 'could',
 'daily',
 'day',
 'deep',
 'deeply',
 'did',
 'dizzy',
 'do',
 'doctor',
 'doe',
 'down',
 'dry',
 'dull',
 'earlier',
 'eat',
 'even',
 'every',
 'everyday',
 'exercise',
 'exercised',
 'exercising',
 'experiencing',
 'extreme',
 'fatigue',
 'fatigued',
 'feel',
 'feeling',
 'fever',
 'few',
 'fire',
 'flu',
 'for',
 'gland',
 'go',
 'going',
 'good',
 'goodbye',
 'ha',
 'hacking',
 'hard',
 'have',
 'having',
 'headache',
 'health',
 'helping',
 'hour',
 'hurt',
 'hurting',
 'i',
 'ill',
 'in',
 'interfere',


In [10]:
# list for training data
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [11]:
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 10
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(output_shape, activation = "softmax"))
initial_learning_rate = 0.01
decay_steps = 10000
decay_rate = 0.9
learning_rate_fn = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps, decay_rate)
adam = tf.keras.optimizers.Adam(learning_rate_fn)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=10, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               26880     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 35,656
Trainable params: 35,656
Non-trainable params: 0
_________________________________________________________________
None
Train on 108 samples
Epoch 1/10
108/108 [==============================] - 1s 7ms/sample - loss: 1.8820 - acc: 0.1667
Epoch 2/10
108/108 [==============================] - 0s 179us/sample - loss: 1.1210 - acc: 0.6944
Epoch 3/10
108/108 [==============================] - 0s 161us/sample - loss: 0.6223 - acc: 0.

In [12]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["intent"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
patterns = ("Welcome to our healthcare app! I'm Mitra, your virtual assistant. How can I help you today?",
        "Hi there! I'm Mitra, your personal healthcare assistant. What can I do for you?",
        "Hello! My name is Mitra, and I'm here to assist you with any healthcare-related questions or concerns you may have. How can I be of service today?",
        "Good day! I'm Mitra, your friendly healthcare chatbot. What brings you here today?"
)
print(random.choice(patterns))
# running the chatbot
while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

Good day! I'm Mitra, your friendly healthcare chatbot. What brings you here today?
I am not feeling good
I'm sorry you're not feeling well. Can you describe your symptoms for me?
I am experiencing back pain
Back pain can be tough to deal with. Have you tried doing some gentle stretching exercises?
Yes but still the pain persists
Great! Are you taking any medication to ease your pain?
Yes, but still there is back pain
Would you like me to schedule an appointment with our doctor?
Yes kindly schedule
Your appointment will be scheduled shortly. Kindly check messages from our end regarding appointment details.
Thank you
Remember to prioritize your health and wellness. If you have any further questions, don't hesitate to ask. Goodbye!
